In [1]:
import cv2
import torch
import requests
import numpy as np
import pandas as pd
import torchvision.models as models
import torchvision.transforms as transforms
from tqdm.notebook import tqdm
from PIL import Image
import os

In [2]:
video_path = r'./training_set/Videos/'
frame_path = r'./training_set/Frames/'
n_frame_path = r'./training_set/N-Frames/'
feature_path = r'./training_set/Features/AlexNetFC7/'

In [3]:
error_log = list()

In [4]:
for filename in os.listdir(feature_path):
    video_name = filename[0:5] #because videos in Features use format xxxxx-xxx.csv
    frame_num = filename[6:9]
    frame_filename = frame_path + video_name + '-' + frame_num + '.png'
    if not os.path.exists(frame_filename):
        cap = cv2.VideoCapture(video_path + str(int(video_name)) + '.mp4')
        cap.set(cv2.CAP_PROP_POS_FRAMES, int(frame_num) - 1)
        res, frame = cap.read()
        if res == 0:
            error_log.append((video_name, frame_num))
        else:
            cv2.imwrite(frame_filename, frame)
        cap.release()

In [5]:
print(error_log)

[]


In [6]:
# Manually iterate through error_log video frames until we get the frame

for video_name, frame_number in error_log:
    cap = cv2.VideoCapture(video_path + str(int(video_name)) + ".mp4")
    i = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if ret == False:
            print(f"Did not find frame {frame_number} of {video_name}")
            break
        if i == int(frame_number):
            cv2.imwrite(frame_path + video_name + '-' + frame_number + '.png', frame)
            break
        i += 1
    cap.release()

# Extract n frame per video

In [7]:
if not os.path.exists(n_frame_path):
    os.mkdir(n_frame_path)

In [8]:
def get_total_frame(video):
    total = 0

    while True:
        (grabbed, frame) = video.read()
        
        if not grabbed:
            break
        
        total += 1
    
    return total

In [9]:
def get_frame_indexes(total_frame, n):
    return [int(i) for i in np.around(np.linspace(0, total_frame, n))]

In [10]:
get_frame_indexes(100, 8) #sanity check to confirm approximately evenly spaced frames

[0, 14, 29, 43, 57, 71, 86, 100]

In [11]:
n = 8 #number of frame we want to extract
error_log_2 = list()

for filename in tqdm(os.listdir(video_path)):
    video_name = filename.split(".")[0].zfill(5)
    cap = cv2.VideoCapture(video_path + filename)

    total_frame = get_total_frame(cap) #total number of frame of this video
    frame_indexes = get_frame_indexes(total_frame, n) #the index of the frame of interest
    
    video_n_frames_folder = f"{n_frame_path}{video_name}"
    if not os.path.exists(video_n_frames_folder):
        os.mkdir(video_n_frames_folder)

    for frame_index in frame_indexes:
        frame_filename = f"{video_n_frames_folder}/{str(frame_index).zfill(3)}.png"

        if os.path.exists(frame_filename): #Already extracted frame, continue to next
            continue
            
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index - 1)
        res, frame = cap.read()
        if res == 0:
            error_log_2.append((video_path+filename, frame_index))
        else:
            cv2.imwrite(frame_filename, frame)

    cap.release()

In [12]:
print(error_log_2)

[('./training_set/Videos/2568.mp4', 185), ('./training_set/Videos/2934.mp4', 160), ('./training_set/Videos/1965.mp4', 187), ('./training_set/Videos/5657.mp4', 130), ('./training_set/Videos/2675.mp4', 144), ('./training_set/Videos/165.mp4', 187), ('./training_set/Videos/4949.mp4', 155), ('./training_set/Videos/4461.mp4', 166), ('./training_set/Videos/2152.mp4', 173), ('./training_set/Videos/2476.mp4', 188)]


In [13]:
for video_file, frame_index in error_log_2:
    cap = cv2.VideoCapture(video_file)
    video_name = video_file.split("/")[-1].split(".")[0].zfill(5)
    
    extracted = False
    while not extracted:
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
        extracted, frame = cap.read()
        frame_index -= 1 # try the previous frame
    
    cv2.imwrite(f"{n_frame_path}{video_name}/{str(frame_index).zfill(3)}.png", frame)